Build credit advisor agent using AI Tools/functions

In [0]:
%pip install unitycatalog-ai[databricks] unitycatalog-langchain[databricks] databricks-langchain

In [0]:
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false

In [0]:
#Build a UCFunctionToolkit with two functions
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from databricks_langchain import UCFunctionToolkit
function_names = [
    f"{catalog}.{db}.explain_model_shap",
    f"{catalog}.{db}.credit_score"
]
toolkit = UCFunctionToolkit(function_names=function_names)
tools = toolkit.tools

In [0]:
# Imports
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from databricks_langchain import UCFunctionToolkit, ChatDatabricks
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
import mlflow

# Initialize the LLM
LLM_ENDPOINT = "databricks-claude-3-7-sonnet"  # or your preferred model
llm = ChatDatabricks(endpoint=LLM_ENDPOINT, temperature=0.0)

# Define a system+human prompt template
prompt = ChatPromptTemplate.from_messages([
("system", """
You are a formal, exploratory Credit Risk Advisor for business users.  
Whenever a customer ID is supplied, follow this exact sequence:

1. **Credit Score:**  
   – Call `credit_score` to fetch the customer’s prediction and full feature record.  
   – Label it **Credit Score:** and summarize in business terms.

2. **SHAP Explanation:**  
   – Call `explain_model_shap` to fetch the customer’s SHAP values.  
   – Label it **SHAP Explanation:** and summarize in business terms.
   – For *each* of the top 5 contributing features, list:  
     • **Feature name**  
     • **SHAP value** (with sign and magnitude)  
     • **Business impact** (“This feature increased/decreased risk because…”)  
     • **Fairness note** (“This feature contributes more to risk for Group A than Group B, increasing bias by X%.”)

3. As a final step, consolidate all the results and findings from the above step and summarize in business-friendly language what the score means and a detailed, explainable, and fairness-aware credit-risk analysis tailored to business users for credit risk management. Show the consolidated summary first, and then the details

If no customer ID is provided, ask explicitly: "A valid customer ID is required to retrieve credit risk insights. Please provide the customer ID."

Always clearly label outputs, explain their meaning in straightforward business language, maintain a formal yet exploratory tone, and highlight potential compliance implications.
"""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Enable MLflow autologging for traceability
mlflow.langchain.autolog()

# Create the agent wired with your tools and prompt
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

# Wrap in an executor for easy programmatic use
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [0]:
#Example invocations:
print(agent_executor.invoke({"input": "what is the default risk prediction for cust_id 5451. explain why the model predicted that value for cust_id 5451."}))